In [1]:
from copy import deepcopy
import subprocess
import numpy as np
import open3d as o3d
import trimesh
import os
import time
print(o3d.__version__)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
0.18.0


In [2]:
def subdivide_surface_fitting(decimated_mesh, target_mesh, iterations=1):
    subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_mesh, number_of_iterations=iterations)
    print(subdivided_mesh)
    subdivided_mesh.compute_vertex_normals()
    
    pcd_target = o3d.geometry.PointCloud()
    pcd_target.points = o3d.utility.Vector3dVector(target_mesh.vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_target)
    subdivided_vertices = np.array(subdivided_mesh.vertices)
    target_vertices = np.array(target_mesh.vertices)
    fitting_vertices = deepcopy(subdivided_vertices)
    
    for i in range(0, len(subdivided_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_vertices[i], 1)
        fitting_vertices[i] = target_vertices[index[0]]

    subdivided_mesh.vertices = o3d.utility.Vector3dVector(fitting_vertices)
    return subdivided_mesh

In [3]:
def read_triangle_mesh_with_trimesh(avatar_name,enable_post_processing=False):
    # EDIT: next 4 lines replace to maintain order even in case of degenerate and non referenced
    # scene_patch = trimesh.load(avatar_name,process=enable_post_processing)
    if enable_post_processing:
        scene_patch = trimesh.load(avatar_name,process=True)
    else:
        scene_patch = trimesh.load(avatar_name,process=False,maintain_order=True) 
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(scene_patch.vertices),
        o3d.utility.Vector3iVector(scene_patch.faces)
    ) 
    if scene_patch.vertex_normals.size:
        mesh.vertex_normals = o3d.utility.Vector3dVector(scene_patch.vertex_normals.copy())
    if scene_patch.visual.defined:
        # either texture or vertex colors if no uvs present.
        if scene_patch.visual.kind == 'vertex':
            mesh.vertex_colors = o3d.utility.Vector3dVector(scene_patch.visual.vertex_colors[:,:3]/255) # no alpha channel support
        elif scene_patch.visual.kind == 'texture':
            uv = scene_patch.visual.uv
            if uv.shape[0] == scene_patch.vertices.shape[0]:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv[scene_patch.faces.flatten()])
            elif uv.shape[0] != scene_patch.faces.shape[0] * 3:
                assert False
            else:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv)
                if scene_patch.visual.material is not None and scene_patch.visual.material.image is not None:
                    if scene_patch.visual.material.image.mode == 'RGB':
                        mesh.textures = [o3d.geometry.Image(np.asarray(scene_patch.visual.material.image))]
                    else:
                        assert False
        else:
            assert False
    return mesh

In [4]:
GoF = 5

In [6]:
load_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh/reference_mesh_{GoF}.obj', enable_post_processing = False)
print(np.array(load_reference_mesh.vertices), load_reference_mesh)

[[ 0.0993192   0.916233   -0.474626  ]
 [ 0.0916947   0.92080802 -0.47391701]
 [ 0.0993192   0.92080802 -0.47630101]
 ...
 [-0.128957    0.92681497  0.0804478 ]
 [ 0.107329    0.300055    0.107815  ]
 [-0.148982    1.08901     0.208021  ]] TriangleMesh with 19998 points and 40000 triangles.


In [8]:
reconstruct_dancer_5 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_005.obj')
original_dancer_5 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0005.obj')
print(reconstruct_dancer_5)
print(original_dancer_5)
reconstruct_dancer_5.compute_vertex_normals()
original_dancer_5.compute_vertex_normals()
o3d.visualization.draw_geometries([reconstruct_dancer_5])

TriangleMesh with 4999 points and 10000 triangles.
TriangleMesh with 19724 points and 39380 triangles.


In [9]:
decimated_reconstructed_5 = o3d.geometry.TriangleMesh.simplify_quadric_decimation(reconstruct_dancer_5, 10000, boundary_weight= 8000)
fitting_mesh_dancer_5 = subdivide_surface_fitting(decimated_reconstructed_5, original_dancer_5, 1)
fitting_mesh_dancer_5.compute_vertex_normals()

o3d.visualization.draw_geometries([fitting_mesh_dancer_5])


TriangleMesh with 19999 points and 40000 triangles.


In [ ]:
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_5.obj', fitting_mesh_dancer_5, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

In [86]:
GoF = 10
for i in range(0, GoF):
    offset = 5
    reconstruct_dancer_i = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_{i+offset:03}.obj')
    original_dancer_i = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0{i+offset:03}.obj')
    #print(reconstruct_dancer_i)
    #print(original_dancer_i)
    reconstruct_dancer_i.compute_vertex_normals()
    original_dancer_i.compute_vertex_normals()
    #o3d.visualization.draw_geometries([reconstruct_dancer_i])
    decimated_reconstructed_i = o3d.geometry.TriangleMesh.simplify_quadric_decimation(reconstruct_dancer_i, 10000, boundary_weight= 8000)
    #print(decimated_reconstructed_i)
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/decimated_reconstructed_{i+offset:03}.obj', decimated_reconstructed_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    fitting_mesh_dancer_i = subdivide_surface_fitting(decimated_reconstructed_i, original_dancer_i, 1)
    #print(fitting_mesh_dancer_i)
    fitting_mesh_dancer_i.compute_vertex_normals()
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_{i+offset:03}.obj', fitting_mesh_dancer_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    #o3d.visualization.draw_geometries([fitting_mesh_dancer_i])

[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19999 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19999 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19999 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19999 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19999 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19999 points and 40000 triangles.
[Open3D WARNING] 

In [5]:
def compute_D1_psnr(original_mesh, decoded_mesh):
    
    original_vertices = np.array(original_mesh.vertices)
    #original_vertices = normalize_vertices(original_vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    #decoded_vertices = normalize_vertices(decoded_vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.linalg.norm(original_vertices[i] - decoded_vertices[index]))
    MSE = MSE / len(original_vertices)
    #print("D1 mse:",MSE)
    aabb = pcd_original.get_axis_aligned_bounding_box()
    min_bound = aabb.get_min_bound()

    max_bound = aabb.get_max_bound()

    signal_peak = np.linalg.norm(max_bound - min_bound)
    #print(signal_peak)
    psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)
    #print(psnr)
    return psnr

In [6]:
def compute_D2_psnr(original_mesh, decoded_mesh):
    decoded_mesh.compute_vertex_normals()
    
    original_vertices = np.array(original_mesh.vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_decoded.normals = o3d.utility.Vector3dVector(decoded_mesh.vertex_normals)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.dot((original_vertices[i] - decoded_vertices[index])[0], np.array(pcd_decoded.normals)[index][0]))
    MSE = MSE / len(original_vertices)
    
    aabb = pcd_original.get_axis_aligned_bounding_box()
    min_bound = aabb.get_min_bound()

    max_bound = aabb.get_max_bound()

    signal_peak = np.linalg.norm(max_bound - min_bound)
    psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)
    
    return psnr

def compute_MSE_RMSE(original_mesh, decoded_mesh):
    
    original_vertices = np.array(original_mesh.vertices)
    #original_vertices = normalize_vertices(original_vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    #decoded_vertices = normalize_vertices(decoded_vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.linalg.norm(original_vertices[i] - decoded_vertices[index]))
    MSE = MSE / len(original_vertices)
    #print("MSE:", MSE)
    RMSE =np.sqrt(MSE)
    
    return np.log10(MSE), np.log10(RMSE)

from scipy.spatial.distance import directed_hausdorff
def compute_Hausdorff(original_mesh, decoded_mesh):
    original_vertices = np.array(original_mesh.vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    hausdorff = directed_hausdorff(original_vertices, decoded_vertices)
    return hausdorff[0] * 1e4


## get error

In [40]:
reference_mesh = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh/reference_mesh_{GoF}.obj')
print(reference_mesh)
decimated_reference_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(reference_mesh, 10000, boundary_weight= 8000)
print(decimated_reference_mesh)
decimated_reference_mesh_vertices = np.array(decimated_reference_mesh.vertices)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', decimated_reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
loaded_decimated_reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
print(loaded_decimated_reference_mesh)
loaded_decimated_reference_mesh_vertices = np.array(loaded_decimated_reference_mesh.vertices)

TriangleMesh with 19998 points and 40000 triangles.
TriangleMesh with 4999 points and 10000 triangles.
TriangleMesh with 4999 points and 10000 triangles.


## get displacements

In [13]:
dataset = 'Mitch'
GoF = 5

In [21]:
fitting_mesh_dancer = []
for i in range(0, GoF):
    offset = 6
    reconstruct_dancer_i = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/deformed_reference_mesh_{i+offset:03}.obj')
    original_dancer_i = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\meshes/mitch_fr0{i+offset:03}.obj')
    print(reconstruct_dancer_i)
    print(original_dancer_i)
    reconstruct_dancer_i.compute_vertex_normals()
    original_dancer_i.compute_vertex_normals()
    #o3d.visualization.draw_geometries([reconstruct_dancer_i])
    fitting_mesh_dancer_i = subdivide_surface_fitting(reconstruct_dancer_i, original_dancer_i, 1)
    #print(np.array(fitting_mesh_dancer_i.triangles))
    fitting_mesh_dancer.append(fitting_mesh_dancer_i)
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/fitting_mesh_{dataset}_{i+offset:03}.obj', fitting_mesh_dancer_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    o3d.visualization.draw_geometries([fitting_mesh_dancer_i])

TriangleMesh with 5002 points and 10000 triangles.
TriangleMesh with 14992 points and 30000 triangles.
TriangleMesh with 20002 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 5002 points and 10000 triangles.
TriangleMesh with 14992 points and 30000 triangles.
TriangleMesh with 20002 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 5002 points and 10000 triangles.
TriangleMesh with 14993 points and 30000 triangles.
TriangleMesh with 20002 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 5002 points and 10000 triangles.
TriangleMesh with 14991 points and 30000 triangles.
TriangleMesh with 20002 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 5002 points and 10000 triangles.
TriangleMesh with 14990 points and 30000 triangles.
TriangleMesh with 20002 points an

In [16]:
o3d.visualization.draw_geometries(fitting_mesh_dancer)

In [82]:
loaded_decimated_reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
print(loaded_decimated_reference_mesh)
loaded_decimated_reference_mesh_vertices = np.array(loaded_decimated_reference_mesh.vertices)

TriangleMesh with 4999 points and 10000 triangles.


In [64]:
print(np.array(loaded_decimated_reference_mesh.triangles))

[[   3    0    2]
 [  34   19   33]
 [  33   22   45]
 ...
 [4950 4890 4877]
 [4920 4950 4877]
 [4879 4846 4937]]


In [19]:
loaded_decimated_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
print(loaded_decimated_reference_mesh)
loaded_decimated_reference_mesh_vertices = np.array(loaded_decimated_reference_mesh.vertices)
#loaded_decimated_reference_mesh_vertices

TriangleMesh with 5002 points and 10000 triangles.


In [92]:
print(np.array(loaded_decimated_reference_mesh.triangles))

[[   0    1    2]
 [   3    4    5]
 [   5    6    7]
 ...
 [4946 4699 4698]
 [4959 4946 4698]
 [1894 1528 2956]]


In [20]:
subdivided_decimated_reference_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(loaded_decimated_reference_mesh, number_of_iterations=1)
print(subdivided_decimated_reference_mesh)
print(np.array(subdivided_decimated_reference_mesh.triangles))
subdivided_decimated_reference_mesh_vertices = np.array(subdivided_decimated_reference_mesh.vertices)
subdivided_decimated_reference_mesh_vertices

TriangleMesh with 20002 points and 40000 triangles.
[[    0  5002  5004]
 [ 5002     1  5003]
 [ 5003     2  5004]
 ...
 [ 6348  1075 15056]
 [15056  2164 19607]
 [ 6348 15056 19607]]


array([[ 0.42324901,  1.14268994, -0.46395099],
       [ 0.404158  ,  1.15783   , -0.45999601],
       [ 0.40882301,  1.17725003, -0.468577  ],
       ...,
       [-0.25049099,  1.264575  ,  0.2881175 ],
       [-0.2466175 ,  1.25975001,  0.300861  ],
       [-0.2415075 ,  1.24099505,  0.290319  ]])

In [24]:
subdivided_decimated_reference_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(loaded_decimated_reference_mesh, number_of_iterations=1)
#print(subdivided_decimated_reference_mesh)
subdivided_decimated_reference_mesh_vertices = np.array(subdivided_decimated_reference_mesh.vertices)
#o3d.visualization.draw_geometries([subdivided_decimated_reference_mesh])
displacements = []
for i in range(0, GoF):
    offset = 6
    fitting_mesh_dancer_i = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/fitting_mesh_{dataset}_{i+offset:03}.obj', enable_post_processing=False)
    #fitting_mesh_dancer_i = fitting_mesh_dancer[i]
    print(fitting_mesh_dancer_i, subdivided_decimated_reference_mesh)
    fitting_mesh_vertices = np.array(fitting_mesh_dancer_i.vertices)
    #print(np.array(fitting_mesh_dancer_i.triangles))
    displacement_i = fitting_mesh_vertices - subdivided_decimated_reference_mesh_vertices
    np.savetxt(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/displacements_{dataset}_{i+offset:03}.txt', displacement_i, fmt='%8f')
    displacements.append(displacement_i)



TriangleMesh with 20002 points and 40000 triangles. TriangleMesh with 20002 points and 40000 triangles.
TriangleMesh with 20002 points and 40000 triangles. TriangleMesh with 20002 points and 40000 triangles.
TriangleMesh with 20002 points and 40000 triangles. TriangleMesh with 20002 points and 40000 triangles.
TriangleMesh with 20002 points and 40000 triangles. TriangleMesh with 20002 points and 40000 triangles.
TriangleMesh with 20002 points and 40000 triangles. TriangleMesh with 20002 points and 40000 triangles.


In [25]:
input_reference_mesh_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/decimated_reference_mesh.obj'
input_decimated_reference_mesh = read_triangle_mesh_with_trimesh(input_reference_mesh_path, enable_post_processing=False)
print(input_decimated_reference_mesh)
np.array(input_decimated_reference_mesh.vertices)

TriangleMesh with 5002 points and 10000 triangles.


array([[ 0.423249,  1.14269 , -0.463951],
       [ 0.404158,  1.15783 , -0.459996],
       [ 0.408823,  1.17725 , -0.468577],
       ...,
       [ 0.329959,  1.65428 , -0.203166],
       [-0.258648,  1.2667  ,  0.292346],
       [ 0.3415  ,  1.34132 , -0.325197]])

In [26]:
output_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/encoded_decimated_reference_mesh.drc'
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_encoder',
                            '-i', input_reference_mesh_path,
                            '-o', output_path,
                            '-qp', str('14'),
                            '-cl', '7'
                            ], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

Encoder options:
  Compression level = 7
  Positions: Quantization = 14 bits

Encoded mesh saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/encoded_decimated_reference_mesh.drc (48 ms to encode).

Encoded size = 18344 bytes

For better compression, increase the compression level up to '-cl 10' .





In [27]:
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_decoder',
                            '-i', output_path,
                            '-o', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/decode_decimated_reference_mesh.obj'
                            ], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

Decoded geometry saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/decode_decimated_reference_mesh.obj (12 ms to decode)




In [28]:
dataset = 'Mitch'

In [29]:
for i in range(0, GoF):
    offset = 6
    displacement = np.loadtxt(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/displacements_{dataset}_{i+offset:03}.txt')
    pcd = o3d.geometry.PointCloud()
    points = displacement
    pcd.points = o3d.utility.Vector3dVector(points)
    print(pcd)
    #o3d.io.write_point_cloud(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.ply', pcd)
    points=np.asarray(pcd.points)
    dtype = o3d.core.float32
    p_tensor = o3d.core.Tensor(points, dtype=dtype)
    pc = o3d.t.geometry.PointCloud(p_tensor)
    o3d.t.io.write_point_cloud(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/dis_{dataset}_{i+offset:03}.ply', pc, write_ascii=True)

PointCloud with 20002 points.
PointCloud with 20002 points.
PointCloud with 20002 points.
PointCloud with 20002 points.
PointCloud with 20002 points.


In [ ]:
dataset = 'Dancer'

In [90]:
import re

qp = 8
times = []
for i in range(0, GoF):
    offset = 6
    result = subprocess.run([
                                r'G:\Github\draco\build\Debug\draco_encoder',
                                #r'G:\Github\draco\buildforSequenceEncoding\Debug\draco_encoder',
                                '-point_cloud',
                                '-i', fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/dis_{dataset}_{i+offset:03}.ply',
                                '-o', fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/dis_{dataset}_{i+offset:03}.drc',
                                '-qp', str(qp),
                                '-cl', '10'
                                ], capture_output=True, text=True)
    print(result.stdout)
    time_pattern = re.compile(r"\((\d+) ms to encode\)")
    match = time_pattern.search(result.stdout)
    if match:
        times.append(int(match.group(1)))
    
    result = subprocess.run([
                                r'G:\Github\draco\build\Debug\draco_encoder',
                                #r'G:\Github\draco\buildforSequenceEncoding\Debug\draco_encoder',
                                '-point_cloud',
                                '-i', fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/dis_{dataset}_{i+offset:03}.ply',
                                '-o', fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/dis_{dataset}_{i+offset:03}_0.drc',
                                '-qp', str(qp),
                                '-cl', '0'
                                ], capture_output=True, text=True)
    print(result.stdout)

        
if times:
    mean_time = sum(times) / len(times)
    print(f"Mean encoding time: {mean_time:.2f} ms")
print(f"Average encoding time for qp {qp}: {mean_time:.2f} seconds\n\n")

Encoder options:
  Compression level = 10
  Positions: Quantization = 8 bits

Encoded point cloud saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/GoF5/dis_Mitch_006.drc (24 ms to encode).

Encoded size = 18071 bytes


Encoder options:
  Compression level = 0
  Positions: Quantization = 8 bits

Encoded point cloud saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/GoF5/dis_Mitch_006_0.drc (5 ms to encode).

Encoded size = 49712 bytes

For better compression, increase the compression level up to '-cl 10' .


Encoder options:
  Compression level = 10
  Positions: Quantization = 8 bits

Encoded point cloud saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/GoF5/dis_Mitch_007.drc (23 ms to encode).

Encoded size = 19197 bytes


Encoder options:
  Compression level = 0
  Positions: Quantization = 8 bits

Encoded point cloud sa

In [91]:
times= []
for i in range(0, GoF):
    offset = 6
    result = subprocess.run([
                                r'G:\Github\draco\build\Debug\draco_decoder',
                                '-i', fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/dis_{dataset}_{i+offset:03}_0.drc',
                                '-o', fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/decoded_{dataset}_{i+offset:03}_displacements.ply'
                                ], capture_output=True, text=True)
    print(result.stdout)
    
    result = subprocess.run([
                                r'G:\Github\draco\build\Debug\draco_decoder',
                                '-i', fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/dis_{dataset}_{i+offset:03}.drc',
                                '-o', fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/decoded_{dataset}_{i+offset:03}_displacements_10.ply'
                                ], capture_output=True, text=True)
    print(result.stdout)
    time_pattern = re.compile(r"\((\d+) ms to decode\)")
    match = time_pattern.search(result.stdout)
    if match:
        times.append(int(match.group(1)))
        
if times:
    mean_time = sum(times) / len(times)
    print(f"Mean encoding time: {mean_time:.2f} ms")
print(f"Average encoding time for qp {qp}: {mean_time:.2f} seconds\n\n")

Decoded geometry saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/GoF5/decoded_Mitch_006_displacements.ply (3 ms to decode)

Decoded geometry saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/GoF5/decoded_Mitch_006_displacements_10.ply (11 ms to decode)

Decoded geometry saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/GoF5/decoded_Mitch_007_displacements.ply (2 ms to decode)

Decoded geometry saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/GoF5/decoded_Mitch_007_displacements_10.ply (11 ms to decode)

Decoded geometry saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Mitch\reference_mesh/GoF5/decoded_Mitch_008_displacements.ply (2 ms to decode)

Decoded geometry saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\

In [32]:
def calculate_bitrate(file_size, duration):
    return file_size * 8 / duration

In [33]:
number_frames = GoF
frame_rate = 30

In [92]:
total_size = 0
offset = 6
for i in range(0, GoF):
    displacement_file_path = fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/dis_{dataset}_{i+offset:03}.drc'
    displacement_file_size = os.path.getsize(displacement_file_path)
    total_size += displacement_file_size
reference_mesh_file_path = fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/encoded_decimated_reference_mesh.drc'
reference_mesh_file_size = os.path.getsize(reference_mesh_file_path)
total_size += reference_mesh_file_size
total_duration = number_frames / frame_rate
overall_bitrate = calculate_bitrate(total_size, total_duration)

print(f"Total Size of {number_frames} DRC Files: {total_size} bytes")
print(f"Overall Bitrate: {overall_bitrate} bits per second")

bitrate_kbps = overall_bitrate / 1000
bitrate_mbps = overall_bitrate / 1000000


print(f"Overall Bitrate: {bitrate_kbps:.2f} Kbps")
print(f"Overall Bitrate: {bitrate_mbps:.2f} Mbps")

Total Size of 5 DRC Files: 113429 bytes
Overall Bitrate: 5444592.0 bits per second
Overall Bitrate: 5444.59 Kbps
Overall Bitrate: 5.44 Mbps


In [93]:
original_displacements = []
decoded_displacements = []
dis_plys= []
for i in range(0, GoF):
    original_displacement = np.loadtxt(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/displacements_{dataset}_{i+offset:03}.txt')
    decoded_displacement = o3d.io.read_point_cloud(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/decoded_{dataset}_{i+offset:03}_displacements.ply')
    dis_ply = o3d.io.read_point_cloud(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/decoded_{dataset}_{i+offset:03}_displacements.ply')
    original_displacements.append(original_displacement)
    decoded_displacements.append(decoded_displacement)
    dis_plys.append(dis_ply)

In [95]:
d1s = []
d2s = []
mses = []
rmses = []
hausdorffs = []
for m in range(0, GoF):
    decode_decimated_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/decode_decimated_reference_mesh.obj', enable_post_processing=False)
    #print(decode_decimated_reference_mesh)
    np.array(decode_decimated_reference_mesh.vertices)
    start = time.time()
    subdivided_decoded_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decode_decimated_reference_mesh, number_of_iterations=1)
    mesh = deepcopy(subdivided_decoded_mesh)
    triangles = deepcopy(mesh.triangles)
    end = time.time()
    print("subdivision time:", end - start)
    #print(subdivided_decoded_mesh)
    input_decimated_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
    subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(input_decimated_reference_mesh, number_of_iterations=1)
    #print(subdivided_mesh)
    original_dancer = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\meshes/mitch_fr0{m+offset:03}.obj')
    #print(original_dancer)
    decoded_mesh_vertices = np.array(decode_decimated_reference_mesh.vertices)
    subdivided_decoded_mesh_vertices = np.array(subdivided_decoded_mesh.vertices)
    
    
    displacement = np.array(decoded_displacements[m].points)
    
    dis_indexer = o3d.geometry.PointCloud()
    dis_indexer.points = o3d.utility.Vector3dVector(original_displacements[m])
    dis_tree = o3d.geometry.KDTreeFlann(dis_indexer)
    
    pcd_indexer = o3d.geometry.PointCloud()
    pcd_indexer.points = o3d.utility.Vector3dVector(subdivided_mesh.vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_indexer)
    
    reordered_vertices = deepcopy(subdivided_decoded_mesh_vertices)
    start = time.time()
    for i in range(0, len(subdivided_decoded_mesh_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_decoded_mesh_vertices[i], 1)
        [j, dis_index, _] = dis_tree.search_knn_vector_3d(original_displacements[m][index[0]], 1)
        #print(displacement[dis_index], original_displacements[index[0]])
        reordered_vertices[i] += displacement[dis_index[0]]
    end = time.time()
    print("rematching time:", end - start)
    reconstruct_mesh = o3d.geometry.TriangleMesh()
    reconstruct_mesh.triangles = subdivided_decoded_mesh.triangles
    reconstruct_mesh.vertices = o3d.utility.Vector3dVector(reordered_vertices)
    reconstruct_mesh.compute_vertex_normals()
    #o3d.visualization.draw_geometries([reconstruct_mesh])
    o3d.io.write_triangle_mesh(fr'G:\PycharmProjects\Mesh_Editing\Results\decode_Ours\{dataset}/GoF{GoF}/decoded_{dataset}_fr0{m+offset:03}.obj', reconstruct_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    d1 = max(compute_D1_psnr(original_dancer, reconstruct_mesh), compute_D1_psnr(reconstruct_mesh, original_dancer))
    print("D1:", d1)
    d1s.append(d1)
    
    d2 = max(compute_D2_psnr(original_dancer, reconstruct_mesh), compute_D2_psnr(reconstruct_mesh, original_dancer))
    print("D2:", d2)
    d2s.append(d2)

    logmse1, logrmse1 = compute_MSE_RMSE(original_dancer, reconstruct_mesh)
    logmse2, logrmse2 = compute_MSE_RMSE(reconstruct_mesh, original_dancer)
    logmse = min(logmse1, logmse2)
    logrmse = min(logrmse1, logrmse2)
    print("log10 of mse:", logmse, ", log10 of rmse:", logrmse)
    mses.append(logmse)
    rmses.append(logrmse)
    
    hausdorff = compute_Hausdorff(original_dancer, reconstruct_mesh)
    print("Hausdorff distance:", hausdorff)
    hausdorffs.append(hausdorff)
o3d.visualization.draw_geometries([reconstruct_mesh])
print("average D1:", np.mean(d1s))
print("average D2:", np.mean(d2s))
print("average log10 of mse:", np.mean(mses))
print("average log10 of rmse:", np.mean(rmses))
print("average Hausdorff:", np.mean(hausdorffs))

subdivision time: 0.0
rematching time: 0.14016199111938477
[Open3D WARNING] Write OBJ can not include triangle normals.
D1: 84.01788265533743
D2: 88.8025602732282
log10 of mse: -7.700294138433431 , log10 of rmse: -3.8501470692167157
Hausdorff distance: 213.41862165001987
subdivision time: 0.0
rematching time: 0.13499927520751953
[Open3D WARNING] Write OBJ can not include triangle normals.
D1: 83.76102306982098
D2: 88.56275786686727
log10 of mse: -7.673375771474228 , log10 of rmse: -3.836687885737114
Hausdorff distance: 213.6462492722064
subdivision time: 0.009032964706420898
rematching time: 0.13258790969848633
[Open3D WARNING] Write OBJ can not include triangle normals.
D1: 82.89282822136646
D2: 87.64026401979585
log10 of mse: -7.586142770621475 , log10 of rmse: -3.7930713853107374
Hausdorff distance: 229.15598506321223
subdivision time: 0.0
rematching time: 0.13524413108825684
[Open3D WARNING] Write OBJ can not include triangle normals.
D1: 81.85710750378462
D2: 86.61999605307776
log

In [90]:
original_displacements = np.loadtxt(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/displacements_{dataset}_005.txt')
decoded_displacements = o3d.io.read_point_cloud(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh\GoF5/decoded_Dancer_005_displacements.ply')
dis_ply = o3d.io.read_point_cloud(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decoded_displacements.ply')
displacement = np.array(decoded_displacements.points)
displacement

[Open3D WARNING] Read PLY failed: unable to open file: G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decoded_displacements.ply


array([[-0.20739794, -0.0242124 , -0.00519706],
       [-0.20542887, -0.0242124 , -0.00716613],
       [-0.20542887, -0.0242124 , -0.00716613],
       ...,
       [-0.02427474,  0.03289054,  0.00464827],
       [-0.02230567,  0.03289054, -0.00125893],
       [ 0.01510659,  0.00729267,  0.00464827]])

In [91]:
decode_decimated_reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decode_decimated_reference_mesh.obj', enable_post_processing=False)
print(decode_decimated_reference_mesh)
np.array(decode_decimated_reference_mesh.vertices)
subdivided_decoded_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decode_decimated_reference_mesh, number_of_iterations=1)
print(subdivided_decoded_mesh)
np.array(subdivided_decoded_mesh.vertices)

TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 19996 points and 40000 triangles.


array([[-0.192079 ,  0.876832 ,  0.039511 ],
       [-0.203194 ,  0.901284 ,  0.047818 ],
       [-0.187633 ,  0.898944 ,  0.048052 ],
       ...,
       [ 0.09304  ,  0.9013425, -0.4686025],
       [ 0.085143 ,  0.904092 , -0.4722295],
       [ 0.0829785,  0.9076015, -0.475271 ]])

In [92]:
decode_decimated_reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decode_decimated_reference_mesh.obj', enable_post_processing=False)
print(decode_decimated_reference_mesh)
np.array(decode_decimated_reference_mesh.vertices)
subdivided_decoded_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decode_decimated_reference_mesh, number_of_iterations=1)
print(subdivided_decoded_mesh)
input_decimated_reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(input_decimated_reference_mesh, number_of_iterations=1)
print(subdivided_mesh)
original_dancer = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0005.obj')
print(original_dancer)
decoded_mesh_vertices = np.array(decode_decimated_reference_mesh.vertices)
subdivided_decoded_mesh_vertices = np.array(subdivided_decoded_mesh.vertices)


displacement = np.array(decoded_displacements.points)

dis_indexer = o3d.geometry.PointCloud()
dis_indexer.points = o3d.utility.Vector3dVector(original_displacements)
dis_tree = o3d.geometry.KDTreeFlann(dis_indexer)

pcd_indexer = o3d.geometry.PointCloud()
pcd_indexer.points = o3d.utility.Vector3dVector(subdivided_mesh.vertices)
pcd_tree = o3d.geometry.KDTreeFlann(pcd_indexer)

reordered_vertices = deepcopy(subdivided_decoded_mesh_vertices)

for i in range(0, len(subdivided_decoded_mesh_vertices)):
    [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_decoded_mesh_vertices[i], 1)
    #print(subdivided_decoded_mesh_vertices[i], np.array(subdivided_mesh.vertices)[index[0]], original_displacements[index[0]])
    [j, dis_index, _] = dis_tree.search_knn_vector_3d(original_displacements[index[0]], 1)
    #print(displacement[dis_index], original_displacements[index[0]])
    reordered_vertices[i] += displacement[dis_index[0]]
    
subdivided_decoded_mesh.vertices = o3d.utility.Vector3dVector(reordered_vertices)

o3d.visualization.draw_geometries([subdivided_decoded_mesh])
d1 = max(compute_D1_psnr(original_dancer, subdivided_decoded_mesh), compute_D1_psnr(subdivided_decoded_mesh, original_dancer))
print("D1:", d1)
d2 = max(compute_D2_psnr(original_dancer, subdivided_decoded_mesh), compute_D2_psnr(subdivided_decoded_mesh, original_dancer))
print("D2:", d2)

TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 20788 points and 39380 triangles.
D1: 53.08561264203229
D2: 63.88097225514851
